<table class="table table-bordered">
    <tr>
        <th style="text-align:center;"><h1>Visual Generative AI Application: Generative Adversarial Networks</h1><h2>Assignment</h2><h3>Specialist Diploma in Applied Generative AI (SDGAI) 
</h3></th>
    </tr>
</table>

# (1) State clearly the goal and objectives you hope to achieve in this notebook


The objective of this project is to design and implement a generative model capable of creating images of fashion items from 10 distinct categories (classes). For this part of the assignment, I will develop a **unconditional (Vanilla) GAN**.

For each model, we will analyze the model performance and tune the model hyperparameters during training phase. For this, we will explore the following:
- Vary the number of epochs:
    - Start with 50
    - Change this to 100
- Change the display step from 500 to 200
- Increase the latent dimension to 200

For each model we will do the following:
1. Load and explore the dataset
2. Build the model
    1. Start with a baseline model
    2. Consider the different hyperparameters that can be tuned
    3. Perform the tuning
    4. Analyse the model's performance based on the different tuning strategies
3. Evaluate the model
    1. Use the model to generate new **specified** images from noise

# (2) Import libraries

In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST 
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torchvision import datasets

import random
import datetime

import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0) 

In [ ]:
device ='cuda' if torch.cuda.is_available else 'cpu'
print(f'Using {device} device')

# (3) Load/Download Dataset  

We will now download the Dataset for this assignment. You may amend the __batch_size__ parameter, __transform__ function or the attributes of the Dataloader as you deem fit for your processing.

In [ ]:
batch_size = 64  # Batch size for training and testing

dataset_path = 'D:\\Users\\ng_a\\My NP SDGAI\\PDC-2\\VGAA\\Assignment\\'  # Use double backslashes

dataset = datasets.FashionMNIST(dataset_path, download=False, transform=transforms.ToTensor())

dataloader = DataLoader(
    # datasets.FashionMNIST(dataset_path, download=True, transform=transforms.ToTensor()),
    dataset,
    batch_size=batch_size,
    shuffle=True
)

# (4) Explore the data

In [ ]:
labels_map = {
    0: 'T-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
}
# Create a subplot with 4x4 grid
fig, axs = plt.subplots(4, 4, figsize=(8, 8))

# Loop through each subplot and plot an image
for i in range(4):
    for j in range(4):
        image, label = dataset[i * 4 + j]  # Get image and label
        image_numpy = image.numpy().squeeze()    # Convert image tensor to numpy array
        axs[i, j].imshow(image_numpy, cmap='gray')  # Plot the image
        axs[i, j].axis('off')  # Turn off axis
        axs[i, j].set_title(f"{labels_map[label]}")  # Set title with label

plt.tight_layout()  # Adjust layout
plt.show()  # Show plot

In [ ]:
def show_tensor_images(image_tensor, num_images=9, size=(1, 28, 28)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in a uniform grid.
    '''

    # Move the image tensor to CPU
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=3)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.axis('off')       
    plt.show()

# (5) Modeling

## (5a) Noise Generator

In [ ]:
def get_noise(n_samples, z_dim, device='cuda'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''

    # Returns the random samples of z_dim dimension
    return torch.randn(n_samples, z_dim, device=device)

## (5b) Generator

In [ ]:
# Defines a single generator block
def get_generator_block(input_dim, output_dim):
    '''
    Function for returning a block of the generator's neural network
    given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a generator neural network layer, with a linear transformation 
          followed by a batch normalization and then a relu activation
    '''
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [ ]:
#  Defines a generator class
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
          (MNIST images are 28 x 28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        super(Generator, self).__init__()

        # Declare your generator block with the following layers:
        # Layer 1: zdim
        # Layer 2: hidden_dim*2
        # Layer 3: hidden_dim*4
        # Layer 4: hidden_dim*8
        # Layer 5: im_dim (Use a sigmoid Activation Function for this layer)
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
        
    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return self.gen(noise)
    
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

## (5c) Discriminator

In [ ]:
#  Defines a discriminator block function
def get_discriminator_block(input_dim, output_dim):
    '''
    Discriminator Block
    Function for returning a neural network of the discriminator given input and output dimensions.
    Parameters:
        input_dim: the dimension of the input vector, a scalar
        output_dim: the dimension of the output vector, a scalar
    Returns:
        a discriminator neural network layer, with a linear transformation 
          followed by an nn.LeakyReLU activation with negative slope of 0.2 
          (https://pytorch.org/docs/master/generated/torch.nn.LeakyReLU.html)
    '''
    return nn.Sequential(
         nn.Linear(input_dim, output_dim), # Layer 1
         nn.LeakyReLU(0.2, inplace=True) # To prevent the "vanishing gradient" problem
    )

In [ ]:
# Defines a Disciminator class
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
            (MNIST images are 28x28 = 784 so that is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=784, hidden_dim=128):
        super(Discriminator, self).__init__()

        # Declare your discriminator block with the following layers:
        # Layer 1: im_dim
        # Layer 2: hidden_dim*8 (Use a dropout with probabilty 0.3)
        # Layer 3: hidden_dim*4 (Use a dropout with probabilty 0.3)
        # Layer 4: hidden_dim*2
        # Layer 5: 1 
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 8),
            nn.Dropout(0.3),
            get_discriminator_block(hidden_dim * 8, hidden_dim * 4),
            nn.Dropout(0.3),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            nn.Linear(hidden_dim * 2, 1)
        )       

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        return self.disc(image)        
    
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc       

# (6) Training, Tuning, Evaluation

In [ ]:
# Define hyperparameters

# Use the criterion BCE with Logits Loss
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 100
display_step = 1000
lr = 0.0002

In [ ]:
# Initialize the generator, discriminator, and optimizers.
def do_create_gen(z_dim, lr, device):
    gen = Generator(z_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr = lr)

    return gen, gen_opt

In [ ]:
# Initialize the discriminator, and optimizers.
def do_create_disc(lr, device):
    disc = Discriminator().to(device) 
    disc_opt = torch.optim.Adam(disc.parameters(), lr = lr)

    return disc, disc_opt

In [ ]:
# Defines generator loss function
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''
    # Create noise vectors. Remember to pass the device argument to the get_noise function.
    fake_noise = get_noise(num_images, z_dim, device=device)
    
    # Generate a batch of fake images
    fake = gen(fake_noise)

    # Get the discriminator's prediction of the fake image.
    disc_fake_pred = disc(fake)

    # Calculate the generator's loss. Remember the generator wants the discriminator to think that its fake images are real
    # *Important*: You should NOT write your own loss function here - use criterion(pred, true)!
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    
    return gen_loss

In [ ]:
# Defines disciminator loss function
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''

    # Create noise vectors and generate a batch (num_images) of fake images. Make sure to pass the device argument to the noise.
    fake_noise = get_noise(num_images, z_dim, device=device)

    # Generate a batch (num_images) of fake images. Make sure to pass the device argument to the noise.
    fake = gen(fake_noise)

    # Get the discriminator's prediction of the fake image 
    disc_fake_pred =disc(fake.detach())

    # Calculate the loss. Don't forget to detach the generator! (Use the 'criterion' function. 
    # You need a 'ground truth' tensor in order to calculate the loss. For example, a ground truth tensor for a fake image is all zeros.)
    # *Important*: You should NOT write your own loss function here - use criterion(pred, true)!
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))

    # Get the discriminator's prediction of the real image and calculate the loss.
    disc_real_pred = disc(real)

    # Calculate the discriminator's loss by averaging the real and fake loss
    # and set it to disc_loss.
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    
    return disc_loss

In [ ]:
def do_train(gen, gen_opt, disc, disc_opt, n_epochs, display_step, criterion, z_dim, device):
    cur_step = 0
    mean_generator_loss = 0
    mean_discriminator_loss = 0
    test_generator = True # Whether the generator should be tested
    gen_loss = False
    error = False

    gen_loss_combine = []
    dis_loss_combine = []
    for epoch in range(n_epochs):
        print('Training at Epoch ', epoch)
        # Dataloader returns the batches
        for real, _ in tqdm(dataloader, total=int(60000/dataloader.batch_size)):
            cur_batch_size = len(real)
            real = real.view(cur_batch_size, -1).to(device)
            disc_opt.zero_grad()
    
            # Calculate discriminator loss
            disc_loss = get_disc_loss(gen, disc, criterion, real, cur_batch_size, z_dim, device)
    
            # Update gradients
            disc_loss.backward(retain_graph=True)
    
            # Update optimizer
            disc_opt.step()
    
            # For testing purposes, to keep track of the generator weights
            # if test_generator:
            # old_generator_weights = gen.gen[0][0].weight.detach().clone()
    
            gen_opt.zero_grad() # Zero out the gradients.
            gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device) # Calculate the generator loss, assigning it to gen_loss.
            gen_loss.backward() # Backprop through the generator: update the gradients and optimizer.
            gen_opt.step()
    
            # Keep track of the average discriminator loss
            mean_discriminator_loss += disc_loss.item() / display_step
    
            # Keep track of the average generator loss
            mean_generator_loss += gen_loss.item() / display_step
    
        print(f"Epoch {epoch}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
        fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        fake = gen(fake_noise)
        show_tensor_images(fake)
        show_tensor_images(real)
        gen_loss_combine.append(mean_generator_loss)
        dis_loss_combine.append(mean_discriminator_loss)
        mean_generator_loss = 0
        mean_discriminator_loss = 0
    return gen_loss_combine, dis_loss_combine

In [ ]:
def plot_eval_curves(dis_loss_combine, gen_loss_combine, title, filename):
    plt.figure()
    plt.plot(np.arange(len(dis_loss_combine)), dis_loss_combine,'r')
    plt.plot(np.arange(len(gen_loss_combine)), gen_loss_combine,'b')
    plt.legend(['Dis Loss','Gen Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(title)
    plt.savefig(filename)

# (6a) Start with 50 Epochs

In [ ]:
n_epochs = 50

gen_1, gen_1_opt = do_create_gen(z_dim, lr, device)
disc_1, disc_1_opt = do_create_disc(lr, device)
gen_loss_combine_1, dis_loss_combine_1 = do_train(gen_1, gen_1_opt, disc_1, disc_1_opt, n_epochs, display_step, criterion, z_dim, device)

In [ ]:
filename = datetime.datetime.now().strftime('%d-%m-%y-%H_%M_unGAN_eval_loss.png')
plot_eval_curves(dis_loss_combine_1, gen_loss_combine_1, 'No. of Epochs = 50', filename)

In [ ]:
print (dataloader)
print (len(dataset))

In [ ]:
print (len(dis_loss_combine_1))
print (dis_loss_combine_1)

print (len(gen_loss_combine_1))
print (gen_loss_combine_1)

## (6b) Tuning: Increase the number of epochs to 100

In [ ]:
# To do
n_epochs = 100

gen_2, gen_2_opt = do_create_gen(z_dim, lr, device)
disc_2, disc_2_opt = do_create_disc(lr, device)
gen_loss_combine_2, dis_loss_combine_2 = do_train(gen_2, gen_2_opt, disc_2, disc_2_opt, n_epochs, display_step, criterion, z_dim, device)

In [ ]:
filename = datetime.datetime.now().strftime('%d-%m-%y-%H_%M_unGAN_eval_loss.png')
plot_eval_curves(dis_loss_combine_2, gen_loss_combine_2, 'No. of Epochs = 100', filename)

## 6(c): Tuning: Reduce the display step to 200

In [ ]:
n_epochs = 100
display_step = 200

gen_3, gen_3_opt = do_create_gen(z_dim, lr, device)
disc_3, disc_3_opt = do_create_disc(lr, device)
gen_loss_combine_3, dis_loss_combine_3 = do_train(gen_3, gen_3_opt, disc_3, disc_3_opt, n_epochs, display_step, criterion, z_dim, device)

In [ ]:
filename = datetime.datetime.now().strftime('%d-%m-%y-%H_%M_unGAN_eval_loss.png')
plot_eval_curves(dis_loss_combine_3, gen_loss_combine_3, 'Display Step = 200', filename)

## 6(d) Tuning: Increase latent dimension to 200

In [ ]:
n_epochs = 100
z_dim = 200
display_step = 1000

gen_4, gen_4_opt = do_create_gen(z_dim, lr, device)
disc_4, disc_4_opt = do_create_disc(lr, device)
gen_loss_combine_4, dis_loss_combine_4 = do_train(gen_4, gen_4_opt, disc_4, disc_4_opt, n_epochs, display_step, criterion, z_dim, device)

In [ ]:
filename = datetime.datetime.now().strftime('%d-%m-%y-%H_%M_unGAN_eval_loss.png')
plot_eval_curves(dis_loss_combine_4, gen_loss_combine_4, 'Latent Dim = 200', filename)

## (7) Evaluation: Find the best

In [ ]:
# import numpy as np
import pandas as pd
from IPython.display import display

# Compute the average losses
avg_gen_loss_1 = round(np.mean(gen_loss_combine_1), 2) # two decimal places
avg_dis_loss_1 = round(np.mean(dis_loss_combine_1), 2)

avg_gen_loss_2 = round(np.mean(gen_loss_combine_2), 2)
avg_dis_loss_2 = round(np.mean(dis_loss_combine_2), 2)

avg_gen_loss_3 = round(np.mean(gen_loss_combine_3), 2)
avg_dis_loss_3 = round(np.mean(dis_loss_combine_3), 2)

avg_gen_loss_4 = round(np.mean(gen_loss_combine_4), 2)
avg_dis_loss_4 = round(np.mean(dis_loss_combine_4), 2)

# Create a DataFrame to store the average losses
loss_data = {
    'Experiment': ['Experiment 1', 'Experiment 2', 'Experiment 3', 'Experiment 4'],
    'Average Generator Loss': [avg_gen_loss_1, avg_gen_loss_2, avg_gen_loss_3, avg_gen_loss_4],
    'Average Discriminator Loss': [avg_dis_loss_1, avg_dis_loss_2, avg_dis_loss_3, avg_dis_loss_4]
}

df = pd.DataFrame(loss_data)

# Display the DataFrame as a table
display(df)

# (6) Generating New Samples From Generative Models

In [ ]:
# Create random noise tensor
num_images = 9
fake_noise = get_noise(num_images, z_dim, device=device)

# Generate fake data from noise
fake = gen_4(fake_noise)

show_tensor_images(fake)

## (7) Saving the trained model

In [ ]:
torch.save(gen_1.state_dict(), 'ungan_generator_1.pth')
torch.save(disc_1.state_dict(), 'ungan_discriminator_1.pth')

torch.save(gen_2.state_dict(), 'ungan_generator_2.pth')
torch.save(disc_2.state_dict(), 'ungan_discriminator_2.pth')

torch.save(gen_3.state_dict(), 'ungan_generator_3.pth')
torch.save(disc_3.state_dict(), 'ungan_discriminator_3.pth')

torch.save(gen_4.state_dict(), 'ungan_generator_4.pth')
torch.save(disc_4.state_dict(), 'ungan_discriminator_4.pth')

# (7) Conclusion

<b>Analysis</b><br>
1. Experiment 1:
    - Generator Loss (2.50): This indicates that the generator is moderately effective at producing realistic data, but there is room for improvement.
    - Discriminator Loss (0.30): A low discriminator loss suggests that the discriminator is quite effective at distinguishing between real and fake data. However, it might be overpowering the generator, leading to a higher generator loss.
2. Experiment 2:
    - Generator Loss (1.84): A lower generator loss compared to Experiment 1 indicates that the generator is performing better and producing more realistic data.
    - Discriminator Loss (0.39): The discriminator loss is slightly higher than in Experiment 1, suggesting a better balance between the generator and discriminator. This balance is crucial for stable GAN training.
3. Experiment 3:
    - Generator Loss (9.37): A very high generator loss indicates that the generator is struggling significantly to produce realistic data. This could be due to various factors such as poor hyperparameter settings or instability in training.
    - Discriminator Loss (1.97): A high discriminator loss suggests that the discriminator is also struggling to distinguish between real and fake data. This indicates overall instability in the GAN training process.
4. Experiment 4:
    - Generator Loss (1.83): Similar to Experiment 2, this low generator loss indicates good performance in generating realistic data.
    - Discriminator Loss (0.38): The discriminator loss is low but balanced with the generator loss, suggesting effective training and a good balance between the generator and discriminator.

<b>Summary</b><br>
- Best Performance: Experiments 2 and 4 show the best performance with low generator losses and balanced discriminator losses. These experiments indicate effective training and a good balance between the generator and discriminator.
- Poor Performance: Experiment 3 shows poor performance with very high losses for both the generator and discriminator, indicating instability and ineffective training.
- Moderate Performance: Experiment 1 shows moderate performance with a higher generator loss and a very low discriminator loss, suggesting that the discriminator might be overpowering the generator.

Overall, Experiments 2 and 4 are the most promising, showing effective training and a good balance between the generator and discriminator. Further fine-tuning of hyperparameters and additional experiments could help improve the performance even more.